# Content-based Filtering

The content-based filtering recommendation algorithm suggests items similar to ones the user has liked before by analyzing the features of the items and the user’s preferences. It compares the content (e.g., product descriptions, keywords, tags) of items with a user’s historical interactions (such as movies they have watched or articles they have read) and suggests items with similar attributes.

Imagine a user watching a sci-fi movie, and the system extracts features like “genre: sci-fi,” “director: Christopher Nolan,” and “themes: space exploration.” The system would then recommend other movies sharing these features, e.g., another sci-fi film with similar themes and directors.

In this notebook, we would take a very simple and straight-forward approach to implement Content-based Filtering. We make recommendations by finding the most similar items to the user's last interacted item, where similarity is defined based on the textual description of the items. The score is the similarity between the target item and the neighbors.

Specifically, we will:
- Select a few description fields from the item metadata
- Transform the features into numerical format, e.g. TFIDF for text data, Multi-Hot Encoders for categorical features
- Concat and normalize the transformation output
- Use cosine similarity to look up neighbor items and use it as the prediction score

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import dill
import mlflow
import numpy as np  # required for the scikit-learn pipeline to work
import pandas as pd
import torch
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler

load_dotenv()

sys.path.insert(0, "..")


from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.model import ContentBased
from src.train_utils import map_indice

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "005-content-based"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-10-18 17:10:09.364 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 005-content-based...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "005-content-based",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/005-content-based",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(train_item_features, device):
    model = ContentBased(item_features=train_item_features, device=device)
    return model

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
# device = 'cpu'
logger.info(f"Using {device} device")

2024-10-18 17:10:09.515 | INFO     | __main__:<module>:7 - Using mps device


In [6]:
with open("../data/item_metadata_pipeline.dill", "rb") as f:
    item_metadata_pipeline = dill.load(f)

# Test implementation

In [7]:
# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
ratings = [1, 4, 5, 3, 2]
timestamps = [0, 1, 2, 3, 4]
main_category = [
    "All Electronics",
    "Video Games",
    "All Electronics",
    "Video Games",
    "Unknown",
]
title = ["All Electronics", "Video Games", "All Electronics", "Video Games", "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.rating_col: ratings,
        args.timestamp_col: timestamps,
        "main_category": main_category,
        "title": title,
        "description": description,
        "categories": categories,
        "price": price,
    }
)
# Drop duplicated item features so that the ContentBased model will fit correctly in terms of index mapping
fit_df = train_df.drop_duplicates(subset=["item_indice"])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]
val_timestamps = [5, 6, 7]
val_main_category = ["All Electronics", "Video Games", "All Electronics"]
val_title = ["All Electronics", "Video Games", "All Electronics"]
val_description = [["Video games blah blah"], [], ["Video games blah blah"]]
val_categories = [["Video Games"], ["Headsets"], ["Video Games"]]
val_price = ["price: 9.99", "14.99", "price: 9.99"]

val_df = pd.DataFrame(
    {
        "user_indice": val_user_indices,
        "item_indice": val_item_indices,
        args.rating_col: val_ratings,
        args.timestamp_col: val_timestamps,
        "main_category": val_main_category,
        "title": val_title,
        "description": val_description,
        "categories": val_categories,
        "price": val_price,
    }
)
val_item_features = item_metadata_pipeline.transform(val_df).astype(np.float32)

In [8]:
n_users = len(set(user_indices))
n_items = len(set(item_indices))

model = init_model(train_item_features, device)

items1 = [1, 2]
items2 = [0, 3]
predictions = model.predict(items1, items2)
print(predictions)

print("\n\n")

users = [0, 1]
anchor_items = [2, 3]
recommendations = model.recommend(users, anchor_items, k=args.top_K)
print(recommendations)

tensor([0.1166, 0.0384], device='mps:0')





Generating Recommendations:   0%|          | 0/2 [00:00<?, ?it/s]

{'user_indice': [0, 0, 0, 0, 1, 1, 1, 1], 'recommendation': [0, 4, 1, 3, 1, 4, 0, 2], 'score': [0.6481854319572449, 0.13637012243270874, 0.03882099688053131, 0.03839520364999771, 0.9612818360328674, 0.3155585825443268, 0.16881369054317474, 0.03839520364999771]}


# Prep data

In [9]:
def get_last_item(df, item_sequence_col="item_sequence"):
    return df.assign(
        last_item_indice=lambda df: df[item_sequence_col].apply(lambda s: s[-1])
    )

In [10]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=1628641464793


In [11]:
train_df = train_df.pipe(get_last_item)
val_df = val_df.pipe(get_last_item)
full_df = pd.concat([train_df, val_df], axis=0)
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-10-18 17:10:10.974 | INFO     | __main__:<module>:9 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [12]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)
full_df = full_df.pipe(map_indice, idm, args.user_col, args.item_col)

In [13]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
0,AGWGLADIA2MYMYSPDA75SN545CKQ,B005UUO9CK,0.0,1536044878471,13310,2437,Video Games,Wipeout 2 - Xbox 360,"[Product Description, Summer thrills and Winte...","[Video Games, Legacy Systems, Xbox Systems, Xb...",21.99,"[1758.0, 4370.0, 3681.0, 1904.0, 3259.0, 1423....",1136.0
1,AFYRLA4FND2RD57F4LCT2JPSJUJQ,B001G6064W,0.0,1600549574482,7718,4374,Video Games,PROTOTYPE - Playstation 3,"[Product Description, You are Alex Mercer, the...","[Video Games, Legacy Systems, PlayStation Syst...",28.0,"[2800.0, 2923.0, 2552.0, 938.0, 3549.0, 3397.0...",2268.0
2,AGEZWX2WLNXUULH5VO67CSMTNCZQ,B00GIK3YAO,0.0,1435176515000,9496,2377,Video Games,PowerA MOGA Hero Power - Electronic Games,"[POWER A MOGA Hero Power - Electronic Games, 1...",[],None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 946.0, 41...",1968.0
3,AHNY6MA63QMAUZISYSW66EURVVMQ,B00DJRLDMU,0.0,1546786802961,12266,1391,Video Games,Tom Clancy’s The Division Underground - Xbox O...,"[In Expansion I, Underground, the factions of ...","[Video Games, Xbox One, Downloadable Content]",None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",1704.0
4,AFAUFOADANHRMH3GR2FA73A6NHIA,B08VFQ3XJX,5.0,1408655680000,4714,1358,Video Games,Final Fantasy X,"[Product Description, Final Fantasy X finally ...","[Video Games, Legacy Systems, PlayStation Syst...",20.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1232.0, 1...",1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340085,AFBYT34RXLYJCATKKN6IZKM5ZFTA,B002I0J4NE,0.0,1378255234000,9508,1027,Video Games,PlayStation Move Starter Bundle,[PlayStation Move Starter Bundle],"[Video Games, Legacy Systems, PlayStation Syst...",135.8,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 1738.0, 3...",3512.0
340086,AE5WZGXHPBTUJYLOHKQMNPWCFCWA,B00KVOVBGM,0.0,1444416370000,5438,2640,Video Games,PlayStation 4 Console - Destiny Bundle [Discon...,"[Product Description, The ultimate Destiny exp...","[Video Games, Game Genre of the Month]",319.98,"[309.0, 98.0, 4507.0, 512.0, 517.0, 34.0, 2068...",2216.0
340087,AFVVEGO2FG5KK7ERTEYKTSWO4ETQ,B00CMQTVUA,0.0,1354291798000,18121,4651,Video Games,Xbox One + Kinect,[Experience greater convenience and control wi...,"[Video Games, Xbox One, Accessories, Accessory...",259.99,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",243.0
340088,AEFJKBRX2TPPMMJ53DKHANKRYDXQ,B08N6NMGNB,5.0,1510350967758,12236,2877,Video Games,Thrustmaster T300 RS - Gran Turismo Edition Ra...,[Works with PS5 games (PS5 games compatibility...,"[Video Games, Legacy Systems, PlayStation Syst...",449.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 272...",3949.0


# Fit

In [14]:
fit_df = train_df.drop_duplicates(subset=[args.item_col])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)
model = init_model(train_item_features, device)

# Predict

In [15]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
374,AH3JPJ5USQUA24SER6SYQRKKU3PQ,B09WJQC91K,3.0,1629021945172,15118,915,Video Games,"8Bitdo Lite Bluetooth Gamepad for Switch Lite,...","[Compatible with Nintendo Switch, Switch Lite ...","[Video Games, Nintendo Switch, Accessories, Co...",24.99,"[-1, -1, -1, 492, 2758, 2394, 3281, 742, 3345,...",4483
955,AH3JPJ5USQUA24SER6SYQRKKU3PQ,B0754LGLFP,0.0,1629021945172,15118,2664,Computers,Redragon K552 Mechanical Gaming Keyboard Rainb...,[Redragon K552 KUMARA 87 Key Rainbow LED Backl...,"[Video Games, PC, Accessories, Gaming Keyboards]",39.99,"[-1, -1, -1, 492, 2758, 2394, 3281, 742, 3345,...",4483


In [16]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
anchor_item_indice = test_df["last_item_indice"].values[0]
item_indice = idm.get_item_index(item_id)
model.predict([item_indice], [anchor_item_indice])

2024-10-18 17:10:13.256 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AH3JPJ5USQUA24SER6SYQRKKU3PQ and parent_asin = B09WJQC91K


tensor([0.0022], device='mps:0')

# Recommend

In [17]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_user_indices = val_df["user_indice"].values

In [18]:
recommendations = model.recommend(val_user_indices, val_anchor_item_indices, args.top_K)

Generating Recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

# Evaluate

## Ranking metrics

In [19]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,7087,134,0.435118,1.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B00DB2BI00
1,7087,1190,0.408683,3.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B01KUP6K3U
2,7087,3135,0.400531,5.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B074CB2RNQ
3,7087,2985,0.342621,7.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B001NJMMHG
4,7087,2037,0.322374,9.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B00DVPMVIU
...,...,...,...,...,...,...
189795,14319,3540,0.073675,192.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B000PS1TG6
189796,14319,1680,0.073205,194.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B0050FDE4E
189797,14319,2746,0.073198,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B000PC4FYK
189798,14319,4475,0.073062,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001EYUPIS


In [20]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
1757,AGDJVPSADB7OTS2CL4TLYQM4CT4A,B087NNPYP3,5.0,1.0
145,AFJKZEYD2VZSI2NO3JZNMA4XX4RA,B09MRM36JJ,4.0,1.0
170,AFBRTNVOROW7UVA66UPX5YCFC6MQ,B00EP2WNKY,3.0,1.0
1729,AGENEGNVXK2GM4YG35LAYXBO527Q,B071Y568B7,4.0,1.0
87,AEL7SJG67X3FAH6U4KLHBOTF6PZA,B001EYUWDG,5.0,1.0
...,...,...,...,...
393,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0051D8PGM,0.0,18.0
1138,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B00800VJWU,0.0,18.0
1845,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B0050SVMYA,0.0,18.0
1677,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0088TN73M,0.0,19.0


In [21]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
175,1416.0,1075.0,0.916135,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B07NQ1B97S,0,NaN
176,1416.0,1075.0,0.916135,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B07NQ1B97S,0,NaN
78,1416.0,607.0,0.797159,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B0045EPEBW,0,NaN
79,1416.0,607.0,0.797159,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B0045EPEBW,0,NaN
84,1416.0,2089.0,0.647437,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B004OL4980,0,NaN
...,...,...,...,...,...,...,...,...
191540,17528.0,2906.0,0.227378,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B008XXLKLE,0,NaN
191633,17528.0,4390.0,0.225699,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0B3DKXT7G,0,NaN
191634,17528.0,4390.0,0.225699,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0B3DKXT7G,0,NaN
191545,17528.0,1540.0,0.225283,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00GM5T8PK,0,NaN


In [22]:
ranking_report = log_ranking_metrics(args, eval_df)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [23]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_item_indices = val_df["item_indice"].values

In [24]:
classifications = (
    model.predict(val_item_indices, val_anchor_item_indices)
    .cpu()
    .detach()
    .numpy()
    .reshape(-1, 1)
)
classifications = MinMaxScaler(feature_range=(0, 1)).fit_transform(classifications)

In [25]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice,classification_proba,label
0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,7087,3330,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[1596, 1047, 3310, 2885, 1862, 1299, 4500, 209...",1404,0.083198,1
1,AFFPVZ3JNCTQIKAK4XK37E2ENWWA,B00HVBPRUO,4.0,1655428133046,4199,3721,Video Games,Gold Wireless Stereo Headset - PlayStation 4,[A Headset for Gamers: Experience everything f...,"[Video Games, PlayStation 4, Accessories, Head...",None,"[-1, -1, 3810, 3019, 1158, 1714, 98, 309, 2291...",3811,0.202658,1
2,AHAIICWIZT6PYSS5QJNFYP6ZXLCA,B001EYUPJW,0.0,1628811542081,18270,3274,Video Games,Def Jam Fight for NY - Gamecube (Gold),"[The ultimate hip-hop fueled fighting game, De...","[Video Games, Legacy Systems, Nintendo Systems...",349.97,"[2245, 705, 3089, 4462, 3922, 2808, 3476, 3558...",4383,0.098958,0
3,AF2AAA4CWRVF2IYVE7WB6OOIEMFA,B072C3VM5F,0.0,1635286957988,10410,4179,Video Games,Far Cry 5 Gold Edition - Xbox One [Digital Code],[],"[Video Games, Xbox One, Games]",None,"[-1, -1, -1, -1, 1690, 431, 720, 742, 1885, 2711]",2711,0.089875,0
4,AFBRTNVOROW7UVA66UPX5YCFC6MQ,B07YBXFDYK,3.0,1636189764550,9102,2986,Video Games,The Evil Within 2 - PlayStation 4,"[From Shinji Mikami, The Evil Within 2 takes t...","[Video Games, PlayStation 4, Games]",20.98,"[-1, -1, -1, -1, 1853, 195, 4616, 1777, 4654, ...",1373,0.103844,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AGNZODFXG6WNNJUAKR3MI42SDG5A,B0B8RG61GK,0.0,1642617264556,10449,3436,Computers,Fast Charging Cable for Switch/Switch Lite/Swi...,"[Dimensions:, Length: 9.8 Feet/3 Meters]","[Video Games, Legacy Systems, Nintendo Systems...",9.89,"[-1, -1, -1, -1, 2074, 1310, 3877, 184, 2952, ...",4478,0.089409,0
1894,AHA6LZWVG2U4WBXNZRWCESNJXNUA,B002JTX87C,0.0,1646645847748,16655,380,Video Games,Scooby Doo! First Frights NDS,"[Product Description, In Scooby-Doo! First Fri...","[Video Games, Legacy Systems, Nintendo Systems...",43.77,"[1208, 1088, 3365, 4287, 3679, 4048, 2351, 197...",3986,0.095090,0
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,11656,4503,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 2159, 3248, 48, 445, 4343, 584, 907, 4155...",877,0.109342,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,1562,4543,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 4507, 1351, 132, 4660, 1979]",1979,0.060573,1


In [26]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

# Clean up

In [27]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/10/18 17:10:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run 005-content-based at: http://localhost:5003/#/experiments/1/runs/787d8978659041ec897d017bbadb84ad.
2024/10/18 17:10:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.
